In [8]:
from langgraph.graph import END, MessagesState, StateGraph
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage


In [ ]:
template = """You are an expert in reviewing code. Your mission is analyze complexity, how to optimize to improve performance and reformat following style guides from the given problem and user's solution
Problem: {problem}
User's solution: {solution}
Answer:"""

problem = """Given an array of integers, return indices of the two numbers such that they add up to a specific target.

You may assume that each input would have exactly one solution, and you may not use the same element twice.

Example:

Given nums = [2, 7, 11, 15], target = 9,

Because nums[0] + nums[1] = 2 + 7 = 9, return [0, 1]."""

solution = """let nums = [3, 3];
let target = 6;

const twoSum = (nums, target) => {
  for(let i = 0; i < nums.length; i++){
    for(let j = i + 1; j < nums.length; j++){
      if(nums[i] + nums[j] === target){
        return [i, j]
      }
    }
  }
  return null;
} 
twoSum(nums, target)"""
import re
class AgentState(MessagesState):
    problem: str
    solution: str
prompt = ChatPromptTemplate.from_template(template)
def extract_message(state: AgentState):
  message = state["messages"][-1].content
  pattern = r"Problem:\s*\{(.*?)\}\s*Solution:\s*\{(.*?)\}"

  match = re.search(pattern, message)

  if match:
      problem = match.group(1)
      solution = match.group(2)
      print("Problem:", problem)
      print("Solution:", solution)
      return {"problem": problem, "solution": solution}
  else:
      return {"problen": "", "solution": ""}
    
def acall_model(state: AgentState):
    
    problem = state["problem"]
    solution = state["solution"]
    
    model = ChatOllama(model="codeqwen")
    review_agent = prompt | model
    response = review_agent.invoke({"problem": problem, "solution": solution})
    return {"messages": [response]}

# Define the graph
agent = StateGraph(MessagesState)
agent.add_node("model", acall_model)
agent.add_node("extract", extract_message)
agent.add_edge("extract", "model")
agent.set_entry_point("extract")
agent.add_edge("model", END)
chatbot = agent.compile(
)

In [ ]:
response = chatbot.invoke([HumanMessage(content=f"Problem: {problem} Solution: {solution}")])
response

KeyError: 'problem'